In [ ]:
# Import the basic libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

# Import select functions from ICESat-2 surface velocity library
from surface_velocity.readers import atl06_to_dict
from surface_velocity.correlation_processing import smooth_and_diff, fill_seg_ids
from surface_velocity.correlation_processing import velocity, time_diff

In [ ]:
# As an example, display the text for one of the IS2_velocity functions
help(velocity)

## Import ATL06 Dictionaries

Eventually, I think that we could make this more seamless, where a user simply inserts the name of a line, then our package looks for appropriate cycles and does the velocity calculation all in one. For now, I am just testing everything and leaving it how it is.

In [ ]:
# read in dictionaries from two different cycles
data_dir = '/Users/benhills/Google Drive File Stream/Shared drives/IceSat2_Surface_Velocity/Shared_Data/FIS_ATL06/'
fn_1 = 'processed_ATL06_20190822153035_08480411_003_01.h5'
D1=atl06_to_dict(data_dir+fn_1,'/gt2l', index=None, epsg=3031)
fn_2 = 'processed_ATL06_20190523195046_08480311_003_01.h5'
D2=atl06_to_dict(data_dir+fn_2,'/gt2l', index=None, epsg=3031)

In [ ]:
plt.figure(figsize=(8,4))

plt.plot(D1['x_atc']/1000.,D1['h_li'],c='indianred')
plt.plot(D2['x_atc']/1000.,D2['h_li'],c='steelblue')
plt.ylabel('Elevation (m)')
plt.xlabel('Along-Track Distance (km)')

plt.tight_layout()

## Preprocessing - Smooth and Differentiate

In [ ]:
x1,h1 = fill_seg_ids(D1['x_atc'],D1['h_li'],D1['segment_id'])
x2,h2 = fill_seg_ids(D2['x_atc'],D2['h_li'],D2['segment_id'])

h1_smooth,dh1 = smooth_and_diff(x1,h1,win=100)
h2_smooth,dh2 = smooth_and_diff(x2,h2,win=100)

# ------------------------------------------

plt.figure(figsize=(8,6))

plt.subplot(211)
plt.tick_params(labelbottom=False,bottom=False)
plt.plot(x1/1000.,h1,c='grey')
plt.plot(x1/1000.,h1_smooth,c='k')
plt.ylabel('Elevation (m)')

plt.subplot(212)
plt.plot(x1/1000.,dh1,c='k')
plt.xlabel('Along-Track Distance (km)')
plt.ylabel('Surface Slopt (m/m)')

plt.tight_layout()

## Velocity Calculation

Lynn did a lot of work here looking for the right search width, segment length, etc. I think that those numbers should be input as defaults into the function eventually, but I couldn't find the optimal numbers that she had. Right now, I have:
- search_width=1000
- segment_length=5000
- dx=20
- corr_threshold=.65

In [ ]:
dt = time_diff(D1,D2)
vel_xs = np.linspace(np.min(x1)+1000,np.max(x1)-1000,1000)
velocities,correlations = velocity(x1,dh1,dh2,dt,vel_xs,search_width=1000,segment_length=5000)

# ------------------------------------------

from matplotlib.gridspec import GridSpec

plt.figure(figsize=(8,4))
gs = GridSpec(2,2)

plt.subplot(gs[0,0])
plt.tick_params(bottom=False,labelbottom=False)
plt.plot(x1/1000.-29000,h1,'.',c='indianred')
plt.plot(x2/1000.-29000,h2,'.',c='steelblue',ms=3)
plt.ylabel('Elevation (m)')
plt.title('ATL06',fontweight='bold')
plt.xlim(80,580)

plt.subplot(gs[1,0])
plt.tick_params(bottom=False,labelbottom=False)
plt.plot(x1/1000.-29000,dh1,'.',c='indianred')
plt.plot(x2/1000.-29000,dh2,'.',c='steelblue',ms=3)
plt.ylim(-.05,.05)
plt.ylabel('Surface Slope (m/m)')
plt.xlim(80,580)

ax5 = plt.subplot(gs[:,1])
plt.plot(vel_xs/1000.-29000,velocities,'.',c='k',label='ATL06')
plt.ylabel('Velocity (m/yr)')
plt.xlabel('Along-Track Distance (km)')
plt.xlim(80,580)
plt.ylim(-500,1500)

plt.tight_layout()